In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from DataReader import DataReader

In [2]:
# read data
dr = DataReader()
dr.read()
X_train,X_val,X_test,Y_train,Y_val = dr.data()

In [4]:
X_train_0 = []
X_train_1 = []

for (x,y) in zip(X_train,Y_train.flatten()):
    if y == 1:
        X_train_1.append(x)
    else:
        X_train_0.append(x)

In [24]:
X_train

array([[-0.4275529 ,  0.99959475, -0.18224008, ...,  0.80645996,
        -1.0148553 ,  1.0148554 ],
       [ 1.1997807 ,  0.99959475, -0.18224008, ...,  0.80645996,
        -1.0148553 ,  1.0148554 ],
       [ 1.6337364 , -1.0004057 , -0.18224008, ..., -1.4553617 ,
        -1.0148553 ,  1.0148554 ],
       ...,
       [-1.3497087 , -1.0004057 , -0.18224008, ..., -1.1073891 ,
         0.98536223, -0.9853622 ],
       [ 0.38611388,  0.99959475, -0.18224008, ...,  0.80645996,
        -1.0148553 ,  1.0148554 ],
       [ 0.38611388, -1.0004057 , -0.18224008, ..., -1.4553617 ,
        -1.0148553 ,  1.0148554 ]], dtype=float32)

In [5]:
X_train_0 = np.array(X_train_0)
X_train_1 = np.array(X_train_1)

In [6]:
mean0 = X_train_0.mean(axis=0)
mean1 = X_train_1.mean(axis=0)

In [7]:
n = X_train.shape[1]

In [17]:
# Compute in-class covariance
cov_0 = np.zeros((n, n))
cov_1 = np.zeros((n, n))

for x in X_train_0:
    cov_0 += np.dot(np.transpose([x - mean0]), [x - mean0]) / X_train_0.shape[0]
for x in X_train_1:
    cov_1 += np.dot(np.transpose([x - mean1]), [x - mean1]) / X_train_1.shape[0]
    

In [18]:
cov = (cov_0 * X_train_0.shape[0] + cov_1 * X_train_1.shape[0]) / (X_train_0.shape[0] + X_train_1.shape[0])

In [25]:
import numpy.linalg

In [31]:
temp0 = X_test - mean0
temp1 = X_test - mean1

u, s, v = np.linalg.svd(cov, full_matrices=False)
inv = np.matmul(v.T * 1 / s, u.T)

w = np.dot(inv, mean0 - mean1)
b =  (-0.5) * np.dot(mean0, np.dot(inv, mean0)) + 0.5 * np.dot(mean1, np.dot(inv, mean1))\
    + np.log(float(X_train_0.shape[0]) / X_train_1.shape[0])


In [37]:
def _predict(X, w, b):
    # This function returns a truth value prediction for each row of X 
    # by rounding the result of logistic regression function.
    return np.round(_f(X, w, b)).astype(np.int)

def _sigmoid(z):
    # Sigmoid function can be used to calculate probability.
    # To avoid overflow, minimum/maximum output value is set.
    return np.clip(1 / (1.0 + np.exp(-z)), 1e-8, 1 - (1e-8))

def _f(X, w, b):
    # This is the logistic regression function, parameterized by w and b
    #
    # Arguements:
    #     X: input data, shape = [batch_size, data_dimension]
    #     w: weight vector, shape = [data_dimension, ]
    #     b: bias, scalar
    # Output:
    #     predicted probability of each row of X being positively labeled, shape = [batch_size, ]
    return _sigmoid(np.matmul(X, w) + b)

    
def _accuracy(Y_pred, Y_label):
    # This function calculates prediction accuracy
    acc = 1 - np.mean(np.abs(Y_pred - Y_label))
    return acc

In [40]:
p = _predict(X_test, w, b)

import pandas as pd
df = pd.DataFrame({'id':list(range(X_test.shape[0])),
                   'label':p.flatten()})

df.to_csv('result.csv',index=None)